In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time
import os

import pymysql.cursors
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
with open("config.json", "r") as f:
    config = json.load(f)

In [3]:
now = datetime.today() - timedelta()
date_end = now - timedelta(days = 80)
date_start = date_end - timedelta(days = 90)

In [4]:
def get_time_point(date_end, date_start):
    """ Get time points from string date
    Args: Argument
        day: string date (str) 
    Return:
        start_stamp, end_stamp
    """
    import datetime
    end_time = datetime.datetime.strptime(date_end, '%Y-%m-%d')
    start_time = datetime.datetime. strptime(date_start,'%Y-%m-%d')
    start_stamp = datetime.datetime.timestamp(start_time)*1000
    end_stamp = datetime.datetime.timestamp(end_time)*1000
    
    return start_stamp, end_stamp

In [5]:
start_stamp, end_stamp = get_time_point(date_end = str(date_end.date()), date_start = str(date_start.date()))

In [6]:
# Connect to the database
connection = pymysql.connect(host=config["pdb_noti_info"]["host"],
                             user=config["pdb_noti_info"]["user"],
                             password=config["pdb_noti_info"]["password"],
                             database=config["pdb_noti_info"]["database"],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

with connection:
    with connection.cursor() as cursor:
        # Read a single record
        sql = f"""SELECT id, valid_from, group_id, data 
        FROM {config["pdb_noti_info"]["table_noti"]} WHERE 1=1 
        AND valid_from >= {start_stamp/1000} 
        AND valid_from <= {end_stamp/1000}"""
        
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)

In [7]:
df['datetime'] = [datetime.fromtimestamp(x) for x in (df['valid_from']).astype(int)]
df['date'] = df.datetime.dt.date

In [8]:
df['data_processed'] = df['data'].str.decode("utf-8")

In [14]:
def converter(data):
    data = json.loads(data)
    body = data['payload']['notification']['body']
    try:
        news_id = data['payload']['data']['customData']['id']
    except:
        news_id = 0
        print("news_id not found!", end="\r")
    return {
        'body': body,
        'news_id': news_id
    }

In [15]:
df['noti_info'] = df['data_processed'].apply(converter)

In [11]:
data = pd.concat([df.drop(['noti_info'],axis = 1),
                  df['noti_info'].apply(pd.Series)], axis = 1)

In [12]:
def jaccard_similarity(A, B):
    #Find intersection of two sets
    nominator = A.intersection(B)

    #Find union of two sets
    denominator = A.union(B)

    #Take the ratio of sizes
    similarity = len(nominator)/len(denominator)
    
    return similarity

In [13]:
data_error = {
    "noti_id": [],
    "news_id": [],
    "news_title (title gốc của bài viết)": [],
    "noti_title (nội dung noti hiển thị)": []
}
THRESHOLD_DIFF = 0.1
count = 0
for i in range(data.shape[0]):
    noti_id = data['id'][i]
    news_id = data['news_id'][i]
    body = data['body'][i]
    try:
        title = get_title(news_id)
    except:
        #print(news_id, body)
        title = "Không tìm thấy"
    
    if title != body:
        title_words = title.split()
        body_words = body.split()
        jaccard_score = jaccard_similarity(set(title_words), set(body_words))
        if jaccard_score < THRESHOLD_DIFF:
            count += 1
            data_error["noti_id"].append(noti_id)
            data_error["news_id"].append(news_id)
            data_error["noti_title (nội dung noti hiển thị)"].append(body)
            data_error["news_title (title gốc của bài viết)"].append(title)

df_error = pd.DataFrame(data=data_error)